In [ ]:
pip install svgwrite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 957.5 kB/s eta 0:00:00


In [ ]:
pip install cairosvg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 622.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
#IMPORTING LIBRARIES
import cv2
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import svgwrite
import cairosvg
import sys
import datetime
import math
import statistics
import collections
import csv
import random
import urllib
import requests
import logging
import re

In [ ]:
#DEFINING UTILITY FUNCTIONS

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs


def parse_path(d):
    command_re = re.compile(r"([A-Za-z])\s*([\d\.+-]+(?:\s+[\d\.+-]+)*)")
    coord_re = re.compile(r"([\d\.+-]+)")

    commands = command_re.findall(d)
    path = []
    for command, coords in commands:
        coords = [float(x) for x in coord_re.findall(coords)]
        if command.isupper():
            path.append([(command, coords[i:i+2]) for i in range(0, len(coords), 2)])
        else:
            if not path:
                raise ValueError("Relative command without previous point")
            prev_point = path[-1][-1][1]
            path.append([(command.upper(), [prev_point[0] + x, prev_point[1] + y]) for x, y in zip(coords[::2], coords[1::2])])
    return path

def read_svg(svg_path):
    tree = ET.parse(svg_path)
    root = tree.getroot()
    namespaces = {'svg': 'http://www.w3.org/2000/svg'}
    paths = []
    for path_elem in root.findall('.//svg:path', namespaces):
        d = path_elem.attrib.get('d', '')
        points = parse_svg_path(d)
        if points.size > 0:
            paths.append([points])
    return paths

def parse_svg_path(d):
    path = parse_path(d)
    points = []
    for segment in path:
        for point in segment:
            points.append([point.real, point.imag])
    return np.array(points)

def plot(paths_XYs, output_svg_path, output_png_path):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    for i, XYs in enumerate(paths_XYs):
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], linewidth=2)
    ax.set_aspect('equal')
    plt.savefig(output_svg_path, format='svg')
    plt.savefig(output_png_path, format='png')
    plt.close()

def polylines2svg(paths_XYs, svg_path):
    W, H = 0, 0
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))
    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)

    dwg = svgwrite.Drawing(svg_path, profile='tiny', shape_rendering='crispEdges')
    group = dwg.g()
    for i, path in enumerate(paths_XYs):
        path_data = []
        for XY in path:
            path_data.append(("M", (XY[0, 0], XY[0, 1])))
            for j in range(1, len(XY)):
                path_data.append(("L", (XY[j, 0], XY[j, 1])))
            if not np.allclose(XY[0], XY[-1]):
                path_data.append(("Z", None))
        group.add(dwg.path(d=path_data, fill='none', stroke='black', stroke_width=2))
    dwg.add(group)
    dwg.save()

    png_path = svg_path.replace('.svg', '.png')
    fact = max(1, 1024 // min(H, W))
    cairosvg.svg2png(url=svg_path, write_to=png_path, parent_width=W, parent_height=H,
                     output_width=fact * W, output_height=fact * H, background_color='white')

In [ ]:
#REGULARISATION

def fit_polygon(n, radius, center=(0, 0)):
    points = []
    for i in range(n):
        angle = 2 * math.pi * i / n
        x = center[0] + radius * math.cos(angle)
        y = center[1] + radius * math.sin(angle)
        points.append((x, y))
    return points


def fit_line(points):
    x = points[:, 0]
    y = points[:, 1]
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def rectangle(width, height, center=(0, 0)):
    half_width = width / 2
    half_height = height / 2
    x = np.array([-half_width, half_width, half_width, -half_width, -half_width])
    y = np.array([-half_height, -half_height, half_height, half_height, -half_height])
    return np.column_stack((x + center[0], y + center[1]))

def fit_circle(points):
    x = points[:, 0]
    y = points[:, 1]
    x_m = np.mean(x)
    y_m = np.mean(y)
    u = x - x_m
    v = y - y_m
    Suu = np.dot(u, u)
    Suv = np.dot(u, v)
    Svv = np.dot(v, v)
    Suuu = np.dot(u, u**2)
    Suvv = np.dot(u, v**2)
    Svvv = np.dot(v, v**2)
    Svvu = np.dot(v, u**2)
    A = np.array([[Suu, Suv], [Suv, Svv]])
    B = np.array([Suuu + Suvv, Svvv + Svvu]) / 2.0
    uc, vc = np.linalg.solve(A, B)
    xc = x_m + uc
    yc = y_m + vc
    R = np.sqrt(uc**2 + vc**2 + (Suu + Svv) / len(x))
    return xc, yc, R

def fit_ellipse(a, b, center=(0, 0), num_points=360):
    theta = np.linspace(0, 2 * math.pi, num_points, endpoint=False)
    x = center[0] + a * np.cos(theta)
    y = center[1] + b * np.sin(theta)
    return np.column_stack((x, y))

In [ ]:
#SYMMETRY DETECTION

def detect_symmetry(points):
    centroid = np.mean(points, axis=0)
    reflected_points = 2 * centroid - points

    differences = np.linalg.norm(points - reflected_points, axis=1)
    symmetry = np.all(differences < 1e-6)

    return symmetry

In [ ]:
#CURVE COMPLETION
import numpy as np
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression

def complete_curve(points, occlusion_type='connected'):
    if occlusion_type == 'connected':
        x = points[:, 0]
        y = points[:, 1]
        f = interp1d(x, y, kind='linear')
        new_x = np.linspace(x[0], x[-1], num=500)
        new_y = f(new_x)
        completed_points = np.column_stack((new_x, new_y))
        return completed_points
    elif occlusion_type == 'disconnected':
        completed_points = handle_disconnected_occlusions(points)
        return completed_points

def handle_disconnected_occlusions(points):
    if len(points) < 2:
        return points

    start_point = points[0]
    end_point = points[-1]

    lin_reg = LinearRegression()
    lin_reg.fit(points[:, 0].reshape(-1, 1), points[:, 1])
    new_x = np.linspace(start_point[0], end_point[0], num=500)
    new_y = lin_reg.predict(new_x.reshape(-1, 1))

    completed_points = np.vstack((points, np.column_stack((new_x, new_y))))
    return completed_points



In [ ]:
#RUNNING FUNCTION

def process_file(input_path):
    paths_XYs = read_csv(input_path)

    for path in paths_XYs:
        for points in path:

            #similarly for other shapes
            m, c = fit_line(points)
            print(f'Line fit: y = {m}x + {c}')
            xc, yc, R = fit_circle(points)
            print(f'Circle fit: center=({xc}, {yc}), radius={R}')

            symmetry = detect_symmetry(points)
            print(f'Symmetry detected: {symmetry}')

            completed_points = complete_curve(points, occlusion_type='connected')
            print(f'Completed curve points: {completed_points}')

path='problems/isolated.csv'
p=process_file(path)

Line fit: y = 0.0016507402994187911x + 185.10020589178106
Circle fit: center=(137.16611517142806, 185.90924932154098), radius=68.18600547236908
Symmetry detected: False
Completed curve points: [[109.14299774 125.58000183]
 [109.14271918 125.57970924]
 [109.14244063 125.57941665]
 [109.14216207 125.57912405]
 [109.14188351 125.57883146]
 [109.14160496 125.57853887]
 [109.1413264  125.57824628]
 [109.14104784 125.57795368]
 [109.14076929 125.57766109]
 [109.14049073 125.5773685 ]
 [109.14021217 125.5770759 ]
 [109.13993361 125.57678331]
 [109.13965506 125.57649072]
 [109.1393765  125.57619813]
 [109.13909794 125.57590553]
 [109.13881939 125.57561294]
 [109.13854083 125.57532035]
 [109.13826227 125.57502776]
 [109.13798372 125.57473516]
 [109.13770516 125.57444257]
 [109.1374266  125.57414998]
 [109.13714804 125.57385739]
 [109.13686949 125.57356479]
 [109.13659093 125.5732722 ]
 [109.13631237 125.57297961]
 [109.13603382 125.57268702]
 [109.13575526 125.57239442]
 [109.1354767  125.57210